In [18]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import os
import copy
import src.util as util

## 1. Load Configuration File

In [19]:
config = util.load_config()


## 2. Data Collection

In [20]:
def read_raw_data(config: dict) -> pd.DataFrame:
    # Create variable to store raw dataset
    raw_dataset = pd.DataFrame()

    # Raw dataset dir
    raw_dataset_dir = config["raw_dataset_dir"]

    # Look and load add CSV files
    for i in tqdm(os.listdir(raw_dataset_dir)):
        raw_dataset = pd.concat([pd.read_csv(raw_dataset_dir + i), raw_dataset])
    
    # Return raw dataset
    return raw_dataset

In [21]:
raw_dataset = read_raw_data(config)

100%|██████████| 1/1 [00:00<00:00,  9.03it/s]


In [22]:
# Check our data
raw_dataset

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Pharmacy-1,Pharmacy,Indonesia,Nusa Barat-14,Nusa Barat,Bali Nusa,BO-10001798,Respi,Respi/A,Respi/ABO-10001798,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Pharmacy-1,Pharmacy,Indonesia,Nusa Barat-14,Nusa Barat,Bali Nusa,CH-10000454,Respi,Respi/B,Respi/BCH-10000454,731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Hospital-1,Hospital,Indonesia,Jawa Barat-144,Jawa Barat,Jawa,LA-10000240,Hyper,Hyper/F,Hyper/FLA-10000240,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Pharmacy-2,Pharmacy,Indonesia,Banten-38,Banten,Jawa,TA-10000577,Respi,Respi/D,Respi/DTA-10000577,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Pharmacy-2,Pharmacy,Indonesia,Banten-38,Banten,Jawa,ST-10000760,Hyper,Hyper/H,Hyper/HST-10000760,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Pharmacy-249,Pharmacy,Indonesia,Banten-34,Banten,Jawa,FU-10001889,Respi,Respi/C,Respi/CFU-10001889,25.2480,3,0.20,4.1028
9990,9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Pharmacy-37,Pharmacy,Indonesia,Jawa Barat-172,Jawa Barat,Jawa,FU-10000747,Respi,Respi/C,Respi/CFU-10000747,91.9600,2,0.00,15.6332
9991,9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Pharmacy-37,Pharmacy,Indonesia,Jawa Barat-172,Jawa Barat,Jawa,PH-10003645,Diabet,Diabet/D,Diabet/DPH-10003645,258.5760,2,0.20,19.3932
9992,9993,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Pharmacy-37,Pharmacy,Indonesia,Jawa Barat-172,Jawa Barat,Jawa,PA-10004041,Hyper,Hyper/G,Hyper/GPA-10004041,29.6000,4,0.00,13.3200


In [23]:
# Save raw dataset to file
util.pickle_dump(raw_dataset, config["raw_dataset_path"])

## 3. Data Validation

In [24]:
# Check data type each variable
raw_dataset.dtypes

Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales            float64
Quantity           int64
Discount         float64
Profit           float64
dtype: object

In [25]:
# Check the range of data for each variable
raw_dataset.describe()

,Row ID,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,229.858001,3.789574,0.156203,28.656896
std,2885.163629,623.245101,2.225110,0.206452,234.260108
min,1.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,22638.480000,14.000000,0.800000,8399.976000


In [26]:
# It will not be affected
raw_dataset.shape

(9994, 20)

### 3.1. Handling Variables Error

#### 3.1.1. Handing Variabel "date"

In [27]:
# Try to cast data in variable date to datetime
#raw_dataset['Order Date'] = pd.to_datetime(raw_dataset['Order Date'])
#raw_dataset['Ship Date'] = pd.to_datetime(raw_dataset['Ship Date'])

#### 3.1.2. Remove Variabel "Row ID"

In [28]:
raw_dataset = raw_dataset.drop("Row ID", axis=1)
raw_dataset = raw_dataset.drop("Order Date", axis=1)
raw_dataset = raw_dataset.drop("Ship Date", axis=1)


#### 3.1.3. Handle Variabel "Quantity"

In [29]:
raw_dataset["Quantity"] = raw_dataset["Quantity"].astype("float64")

In [30]:
# Check the result
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Order ID       9994 non-null   object 
 1   Ship Mode      9994 non-null   object 
 2   Customer ID    9994 non-null   object 
 3   Customer Name  9994 non-null   object 
 4   Segment        9994 non-null   object 
 5   Country        9994 non-null   object 
 6   City           9994 non-null   object 
 7   State          9994 non-null   object 
 8   Region         9994 non-null   object 
 9   Product ID     9994 non-null   object 
 10  Category       9994 non-null   object 
 11  Sub-Category   9994 non-null   object 
 12  Product Name   9994 non-null   object 
 13  Sales          9994 non-null   float64
 14  Quantity       9994 non-null   float64
 15  Discount       9994 non-null   float64
 16  Profit         9994 non-null   float64
dtypes: float64(4), object(13)
memory usage: 1.3+ MB


In [31]:
# all data types are appropriate, we can save them so that they can be used again later
util.pickle_dump(raw_dataset, config["cleaned_raw_dataset_path"])

## 4. Data Defense

In [32]:
def check_data(input_data, params):
    # Check data types
    #assert input_data.select_dtypes("datetime").columns.to_list() == params["datetime_columns"], "an error occurs in datetime column(s)."
    assert input_data.select_dtypes("object").columns.to_list() == params["object_columns"], "an error occurs in object column(s)."
    assert input_data.select_dtypes("float").columns.to_list() == params["float64_columns"], "an error occurs in float64 column(s)."

    # Check range of data
    assert input_data.Quantity.between(params["range_Quantity"][0], params["range_Quantity"][1]).sum() == len(input_data), "an error occurs in Quantity range."
    assert input_data.Sales.between(params["range_Sales"][0], params["range_Sales"][1]).sum() == len(input_data), "an error occurs in Sales range."
    assert input_data.Discount.between(params["range_Discount"][0], params["range_Discount"][1]).sum() == len(input_data), "an error occurs in Discount range."
    assert input_data.Profit.between(params["range_Profit"][0], params["range_Profit"][1]).sum() == len(input_data), "an error occurs in Profit range."

In [33]:
check_data(raw_dataset, config)

## 5. Data Splitting

In [34]:
# Split input/variable/feature with target/labet/output
x = raw_dataset[config["predictors"]].copy()
y = raw_dataset.Profit.copy()

In [35]:
config["predictors"]

['Ship Mode',
 'Customer ID',
 'Customer Name',
 'Segment',
 'Country',
 'City',
 'State',
 'Region',
 'Product ID',
 'Category',
 'Sub-Category',
 'Product Name',
 'Sales',
 'Quantity',
 'Discount']

In [36]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Ship Mode      9994 non-null   object 
 1   Customer ID    9994 non-null   object 
 2   Customer Name  9994 non-null   object 
 3   Segment        9994 non-null   object 
 4   Country        9994 non-null   object 
 5   City           9994 non-null   object 
 6   State          9994 non-null   object 
 7   Region         9994 non-null   object 
 8   Product ID     9994 non-null   object 
 9   Category       9994 non-null   object 
 10  Sub-Category   9994 non-null   object 
 11  Product Name   9994 non-null   object 
 12  Sales          9994 non-null   float64
 13  Quantity       9994 non-null   float64
 14  Discount       9994 non-null   float64
dtypes: float64(3), object(12)
memory usage: 1.1+ MB


In [37]:
y.value_counts()

0.0000     65
6.2208     43
9.3312     38
5.4432     32
3.6288     32
           ..
83.2508     1
16.1096     1
7.1988      1
1.6510      1
72.9480     1
Name: Profit, Length: 7287, dtype: int64

In [38]:
# First split, splitting train and test set with ratio 0.7:0.3 and do stratify splitting
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 123)

In [39]:
# Second split, splitting test and valid set with ratio 0.5:0.5 and do stratify splitting
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state = 123)

In [40]:
util.pickle_dump(x_train, config["train_set_path"][0])
util.pickle_dump(y_train, config["train_set_path"][1])

util.pickle_dump(x_valid, config["valid_set_path"][0])
util.pickle_dump(y_valid, config["valid_set_path"][1])

util.pickle_dump(x_test, config["test_set_path"][0])
util.pickle_dump(y_test, config["test_set_path"][1])